In [2]:
import random
import json
import numpy as np
import pickle

import nltk
#nltk.download('punkt')
#nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Activation,Dropout
from tensorflow.keras.optimizers import SGD

In [10]:
#!pip install nltk
#nltk.download('omw-1.4')

In [11]:
lemmatizer = WordNetLemmatizer()

In [12]:
intents = json.loads(open("intents.json").read())

In [13]:
words=[]
classes=[]
documents=[]
ignore_letters=["?","!",".",","]

In [14]:
for intent in intents["intents"]:
    for pattern in intent["patterns"]:
        word_list=nltk.word_tokenize(pattern)
        words.extend(word_list)
        documents.append((word_list,intent["tag"]))
        if intent["tag"] not in classes:
            classes.append(intent["tag"])
print(documents)

[(['hello'], 'greetings'), (['hey'], 'greetings'), (['hi'], 'greetings'), (['good', 'day'], 'greetings'), (['Greetengs'], 'greetings'), (['what', "'s", 'up'], 'greetings'), (['how', 'is', 'it', 'going', '?'], 'greetings'), (['yavuz'], 'goodbye'), (['see', 'you', 'later'], 'goodbye'), (['I', 'am', 'Leaving'], 'goodbye'), (['have', 'a', 'Good', 'day'], 'goodbye'), (['bye'], 'goodbye'), (['cao'], 'goodbye'), (['see', 'yaa'], 'goodbye'), (['How', 'old', '?'], 'age'), (['How', 'old', 'are', 'you', '?'], 'age'), (['What', 'is', 'your', 'age', '?'], 'age'), (['age', '?'], 'age'), (['What', 'is', 'your', 'name', '?'], 'name'), (['What', 'should', 'I', 'call', 'you'], 'name'), (['what', "'s", 'your', 'name', '?'], 'name'), (['who', 'are', 'you'], 'name'), (['Can', 'you', 'tell', 'me', 'your', 'name', '?'], 'name'), (['When', 'are', 'you', 'guys', 'open'], 'hours'), (['hey'], 'hours'), (['What', 'are', 'your', 'hours'], 'hours'), (['hours', 'of', 'open'], 'hours')]


In [15]:
words = [lemmatizer.lemmatize(word) for word in words if word not in ignore_letters]

In [16]:
words = sorted(set(words))
classes = sorted(set(classes))

In [17]:
pickle.dump(words, open("words.pkl","wb"))
pickle.dump(classes, open("classes.pkl","wb"))

In [18]:
training = []
output_empty = [0]*len(classes)

In [19]:
for document in documents:
    bag = []
    word_patterns= document[0]
    word_patterns= [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)
    output_row = list(output_empty)
    output_row[classes.index(document[1])]=1
    training.append([bag,output_row])
    

In [20]:
random.shuffle(training)
training = np.array(training)
train_x = list(training[:,0])
train_y = list(training[:,1])

<ipython-input-20-8e8c1affa596>:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


In [21]:
model = Sequential()
model.add(Dense(128,input_shape=(len(train_x[0]),),activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(64,activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]),activation="softmax"))

sgd = SGD(lr=0.01,decay=1e-6,momentum=0.9,nesterov=True)
model.compile(loss="categorical_crossentropy",optimizer=sgd,metrics=["accuracy"])



/home/yavuz/.local/lib/python3.8/site-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


In [22]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               5760      
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 5)                 325       
                                                                 
Total params: 14,341
Trainable params: 14,341
Non-trainable params: 0
_________________________________________________________________


In [23]:
model.fit(np.array(train_x),np.array(train_y),epochs=200,batch_size=5,verbose=1)
print("Done")

Epoch 1/200
6/6 [==============================] - 1s 3ms/step - loss: 1.6198 - accuracy: 0.1481
Epoch 2/200
6/6 [==============================] - 0s 3ms/step - loss: 1.6517 - accuracy: 0.3333
Epoch 3/200
6/6 [==============================] - 0s 3ms/step - loss: 1.6103 - accuracy: 0.2222
Epoch 4/200
6/6 [==============================] - 0s 3ms/step - loss: 1.5884 - accuracy: 0.2593
Epoch 5/200
6/6 [==============================] - 0s 4ms/step - loss: 1.5277 - accuracy: 0.3704
Epoch 6/200
6/6 [==============================] - 0s 3ms/step - loss: 1.4402 - accuracy: 0.2963
Epoch 7/200
6/6 [==============================] - 0s 4ms/step - loss: 1.4305 - accuracy: 0.4444
Epoch 8/200
6/6 [==============================] - 0s 3ms/step - loss: 1.3044 - accuracy: 0.4444
Epoch 9/200
6/6 [==============================] - 0s 3ms/step - loss: 1.2757 - accuracy: 0.5185
Epoch 10/200
6/6 [==============================] - 0s 4ms/step - loss: 1.2081 - accuracy: 0.5926
Epoch 11/200
6/6 [===========

6/6 [==============================] - 0s 3ms/step - loss: 0.0828 - accuracy: 0.9630
Epoch 168/200
6/6 [==============================] - 0s 3ms/step - loss: 0.1213 - accuracy: 0.8889
Epoch 169/200
6/6 [==============================] - 0s 3ms/step - loss: 0.0859 - accuracy: 0.9630
Epoch 170/200
6/6 [==============================] - 0s 3ms/step - loss: 0.0734 - accuracy: 0.9630
Epoch 171/200
6/6 [==============================] - 0s 3ms/step - loss: 0.0986 - accuracy: 0.9259
Epoch 172/200
6/6 [==============================] - 0s 3ms/step - loss: 0.0748 - accuracy: 0.9630
Epoch 173/200
6/6 [==============================] - 0s 2ms/step - loss: 0.0661 - accuracy: 1.0000
Epoch 174/200
6/6 [==============================] - 0s 3ms/step - loss: 0.0424 - accuracy: 1.0000
Epoch 175/200
6/6 [==============================] - 0s 3ms/step - loss: 0.1238 - accuracy: 0.9630
Epoch 176/200
6/6 [==============================] - 0s 4ms/step - loss: 0.1239 - accuracy: 0.9630
Epoch 177/200
6/6 [=====

In [24]:
model.save("chatbot_model.h5")